### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Расмотрим пример на датасете из репозитория UCI

**Distinguish between the presence and absence of cardiac arrhythmia and classify it in one of the 16 groups.**

Описание данных - https://archive.ics.uci.edu/ml/datasets/Arrhythmia

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../../data/ml_in_business/arrhythmia/arrhythmia.data", header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10


In [2]:
df.iloc[:,279].nunique(), np.sort(df.iloc[:,279].unique())

(13, array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 15, 16], dtype=int64))

У признака 16 классов классификации (номер признака, название, количество):
- 01 - Normal				          245
- 02 - Ischemic changes (Coronary Artery Disease)   44
- 03 - Old Anterior Myocardial Infarction           15
- 04 - Old Inferior Myocardial Infarction           15
- 05 - Sinus tachycardy			           13
- 06 - Sinus bradycardy			           25
- 07 - Ventricular Premature Contraction (PVC)       3
- 08 - Supraventricular Premature Contraction	    2
- 09 - Left bundle branch block 		            9	
- 10 - Right bundle branch block		           50
- 11 - 1. degree AtrioVentricular block	            0	
- 12 - 2. degree AV block		            0
- 13 - 3. degree AV block		            0
- 14 - Left ventricule hypertrophy 	            4
- 15 - Atrial Fibrillation or Flutter	            5
- 16 - Others				           22

**Будем распознавать класс 2 с ишемическими изменениями. Остальное приравняем к норме.**

In [3]:
data = df.copy()
data[279].replace([1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 0, inplace=True)
data[279].replace(2, 1, inplace=True)
data[279]

0      0
1      0
2      0
3      0
4      0
      ..
447    0
448    0
449    1
450    0
451    0
Name: 279, Length: 452, dtype: int64

У нас есть 280 признаков и 1 целевая переменная (бинарная) - нужно определить есть ишемические изменения или нет.

In [4]:
print(data.shape)

(452, 280)


Всего 452 объекта

Посмотрим на соотношение классов

In [5]:
data.iloc[:, -1].value_counts()

0    408
1     44
Name: 279, dtype: int64

In [6]:
def get_num_features(df):
    """
    функция возвращает список числовых признаков
    :param df
    :return: list
    """
    numerical_features = df.select_dtypes(include=[np.number])
    # print(f"count of numeric_features_train {numerical_features.shape[1]}")
    return numerical_features.columns


def get_cat_features(df):
    """
    функция возвращает список признаков с типом "объект"
    :param df:
    :return: list
    """
    categorical_features = df.select_dtypes(include=[np.object])
    return categorical_features.columns

Числовые принаки

In [7]:
get_num_features(data)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            270, 271, 272, 273, 274, 275, 276, 277, 278, 279],
           dtype='int64', length=275)

Категориальные признаки

In [8]:
get_cat_features(data)

Int64Index([10, 11, 12, 13, 14], dtype='int64')

In [9]:
data[[10, 11, 12, 13, 14]].dtypes

10    object
11    object
12    object
13    object
14    object
dtype: object

Посмотрим на категориальные признаки повнимательнее. Видим, что все ои числовые, но представлены в строковм выражении из-за того, что пропуски заполнены "?".

In [10]:
data[10].unique()

array(['13', '37', '34', '11', '66', '49', '7', '69', '71', '42', '51',
       '20', '45', '75', '-24', '28', '39', '78', '56', '10', '17', '112',
       '52', '48', '153', '172', '16', '32', '46', '23', '35', '3', '-8',
       '-5', '4', '60', '8', '-18', '76', '64', '9', '79', '33', '14',
       '30', '53', '25', '62', '?', '43', '31', '19', '50', '68', '-164',
       '-56', '15', '-2', '87', '-136', '175', '-124', '142', '-13', '-9',
       '58', '47', '-165', '73', '21', '38', '70', '-92', '143', '22',
       '72', '57', '41', '65', '36', '63', '55', '44', '77', '24', '81',
       '12', '29', '18', '40', '54', '-1', '104', '-167', '-15', '67',
       '108', '-75', '160', '114', '-22', '120', '-115', '147', '-82',
       '-63', '-172', '102', '179', '86', '-52', '-49', '174', '-27',
       '95', '-123', '61', '137', '164', '5', '-177', '-103', '-106',
       '-42', '117', '127', '-160', '122', '59', '-16', '26', '100', '0',
       '-21', '-121', '-174', '-51', '105', '-116', '-4', '

Заменяем знаки вопроса на мединные значения по признаку

In [11]:
for col in [10, 11, 12, 13, 14]:
    # извлекаем индексы по условию (тех элементов, которые имеют значение "?")
    indices = np.where(data[col] != '?')
    # получаем спиcок элементов по индексам [0], приводим к строке, к целому и находим медиану
    col_med = np.median(np.take(data[col], indices[0]).astype(str).astype(int))
    # заменяем некорректное значение медианой
    data[col] = data[col].replace('?', col_med)

Проверям что "?" заменились на медиану.

In [12]:
data[10].unique()

array(['13', '37', '34', '11', '66', '49', '7', '69', '71', '42', '51',
       '20', '45', '75', '-24', '28', '39', '78', '56', '10', '17', '112',
       '52', '48', '153', '172', '16', '32', '46', '23', '35', '3', '-8',
       '-5', '4', '60', '8', '-18', '76', '64', '9', '79', '33', '14',
       '30', '53', '25', '62', 41.0, '43', '31', '19', '50', '68', '-164',
       '-56', '15', '-2', '87', '-136', '175', '-124', '142', '-13', '-9',
       '58', '47', '-165', '73', '21', '38', '70', '-92', '143', '22',
       '72', '57', '41', '65', '36', '63', '55', '44', '77', '24', '81',
       '12', '29', '18', '40', '54', '-1', '104', '-167', '-15', '67',
       '108', '-75', '160', '114', '-22', '120', '-115', '147', '-82',
       '-63', '-172', '102', '179', '86', '-52', '-49', '174', '-27',
       '95', '-123', '61', '137', '164', '5', '-177', '-103', '-106',
       '-42', '117', '127', '-160', '122', '59', '-16', '26', '100', '0',
       '-21', '-121', '-174', '-51', '105', '-116', '-4', 

In [13]:
#data = data.astype(str).astype(float)


Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [14]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [15]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

C:\Users\Alex\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.10, 11, 12, 13, 14

Проверяем качество

In [ ]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [ ]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [ ]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

* We now have just 153 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

* Recall that col 4 still holds the actual label

In [ ]:
mod_data.head(10)

Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [ ]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [ ]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [ ]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь: